### Asistant Agent

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken 
from autogen_ext.models.openai import OpenAIChatCompletionClient


In [2]:
# Define a tool that searches the web for information
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."

model = OpenAIChatCompletionClient(model="gpt-4o")


In [3]:
agent = AssistantAgent(
    name="assistant",
    model_client=model,
    tools=[web_search],
    reflect_on_tool_use=True,
    system_message="Use tools to solve tasks"
)

In [4]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="Find information on AutoGen", source="user")],
        cancellation_token=CancellationToken())
    print(response.inner_messages)
    print(response.chat_message)

In [5]:
await assistant_run()

[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=17), content=[FunctionCall(id='call_73VZgHMYJD2Bfa1PqFHVv9mI', arguments='{"query":"AutoGen information"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_73VZgHMYJD2Bfa1PqFHVv9mI')], type='ToolCallExecutionEvent')]
source='assistant' models_usage=RequestUsage(prompt_tokens=60, completion_tokens=32) content='AutoGen is a programming framework designed for creating multi-agent applications. If you need more specific information or details about its features and usage, let me know!' type='TextMessage'


In [6]:
async def assistant_run_stream() -> None:
    await Console(
        agent.on_messages_stream(
            [TextMessage(content="Find information on AutoGen",source="user")],
            cancellation_token=CancellationToken()
        )
    )

In [7]:
await assistant_run_stream()

---------- assistant ----------
[FunctionCall(id='call_d7P2t2tRqsF9LvbX4u5nT0VV', arguments='{"query":"AutoGen"}', name='web_search')]
---------- assistant ----------
[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_d7P2t2tRqsF9LvbX4u5nT0VV')]
---------- assistant ----------
AutoGen is a programming framework used for constructing multi-agent applications. If you have any specific questions about it or need more detailed information, feel free to ask!


### Langchain Tools

In [8]:
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool

In [9]:
df = pd.read_csv("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")
tool = LangChainToolAdapter(PythonAstREPLTool(locals={"df": df}))
model_client = OpenAIChatCompletionClient(model="gpt-4o")
agent = AssistantAgent(
    "assistant",
    tools=[tool],
    model_client=model_client,
    system_message="Use the 'df' variable to access the dataset."
)

In [10]:
await Console(
    agent.on_messages_stream(
        [TextMessage(content="What's the average age of the passengers?", source="user")],
        CancellationToken())
)

---------- assistant ----------
[FunctionCall(id='call_srAKQj23KJSZ23p7no7T8RVg', arguments='{"query":"df[\'Age\'].mean()"}', name='python_repl_ast')]
---------- assistant ----------
[FunctionExecutionResult(content='29.69911764705882', call_id='call_srAKQj23KJSZ23p7no7T8RVg')]
---------- assistant ----------
29.69911764705882


Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content='29.69911764705882', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=111, completion_tokens=22), content=[FunctionCall(id='call_srAKQj23KJSZ23p7no7T8RVg', arguments='{"query":"df[\'Age\'].mean()"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='29.69911764705882', call_id='call_srAKQj23KJSZ23p7no7T8RVg')], type='ToolCallExecutionEvent')])

### Structured Output

In [11]:
from typing import Literal
from pydantic import BaseModel

In [12]:
class AgentResponse(BaseModel):
    thought: str
    response: Literal["happy","sad","neutral"]

In [13]:
# Create an agent that uses the OpenAI GPT-4o model with the custom response format.
model_client = OpenAIChatCompletionClient(model="gpt-4o",response_format=AgentResponse)

In [14]:
agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    system_message="Categorize the input as happy,sad, or neutral following the JSON format.",
)

In [15]:
await Console(agent.run_stream(task="I am happy"))

---------- user ----------
I am happy
---------- assistant ----------
{"thought":"The user expressed a clear positive emotion by explicitly stating their happiness.","response":"happy"}


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='I am happy', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=86, completion_tokens=21), content='{"thought":"The user expressed a clear positive emotion by explicitly stating their happiness.","response":"happy"}', type='TextMessage')], stop_reason=None)

### UserProxyAgent

In [16]:
import asyncio
from autogen_core import CancellationToken
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.messages import TextMessage

In [17]:
async def simple_user_agent():
    agent = UserProxyAgent("user_proxy")
    response = await asyncio.create_task(
        agent.on_messages(
            [TextMessage(content="What is your Name? ",source="user")],
            cancellation_token=CancellationToken(),
        )
    )
    print(f"Your name is {response.chat_message.content}")
    

In [20]:
await simple_user_agent()

Enter your response:  joni


Your name is joni


### VideoSurfer

In [21]:
import asyncio
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.agents import UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.agents.video_surfer import VideoSurfer

In [22]:
async def main() -> None:
    """Main function to run the video agent.
    """
    model_client = OpenAIChatCompletionClient(model="gpt-4o-2024-08-06")

    # Define an agent
    video_agent = VideoSurfer(
        name="VideoSurfer",
        model_client=model_client
    )

    web_surfer_agent = UserProxyAgent(
        name="User"
    )

    # Define a team
    agent_team = MagenticOneGroupChat([web_surfer_agent, video_agent],  model_client=model_client,)

    # Run the team and stream messages to the console
    stream = agent_team.run_stream(task="Find a latest video about magnestic one on youtube and extract quotes from it that make sense.")
    await Console(stream)


In [ ]:
await main()

---------- user ----------
Find a latest video about magnestic one on youtube and extract quotes from it that make sense.
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

Find a latest video about magnestic one on youtube and extract quotes from it that make sense.


To answer this request we have assembled the following team:

User: A human user
VideoSurfer: An agent that can answer questions about a local video.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
   - The request involves finding a video about "magentic one" on YouTube.
   - The task includes extracting quotes from that video.

2. FACTS TO LOOK UP
   - Whether there is a company, product, or topic specifically known as "magentic one." This might be found via a search engine or directly on YouTube.
   - Identify the latest available video related to "magentic one" on YouTube.

3. FACTS TO DERIVE
   - Determine what aspects of the content could c

Enter your response:  https://www.youtube.com/watch?v=23urWKmHS6o


---------- User ----------
https://www.youtube.com/watch?v=23urWKmHS6o
---------- MagenticOneOrchestrator ----------
Please analyze the provided video URL 'https://www.youtube.com/watch?v=23urWKmHS6o' and extract quotes that are meaningful and relevant.
---------- VideoSurfer ----------
I am unable to directly access or download YouTube videos. You will need to download the video to your local system first. Once you have the video file available locally, please provide the file path so that I can proceed with the analysis and quote extraction.
---------- MagenticOneOrchestrator ----------
Please download the video from 'https://www.youtube.com/watch?v=23urWKmHS6o' onto your local system, and then provide the local file path for analysis.


Enter your response:  using mmped


---------- User ----------
using mmped
---------- MagenticOneOrchestrator ----------
Please provide the local file path of the video once you have downloaded it for analysis.
---------- VideoSurfer ----------
Please provide the local file path for the video you have downloaded, so I can proceed with transcribing and extracting quotes from it.
---------- MagenticOneOrchestrator ----------
Please provide the local file path for the video you have downloaded so I can analyze it and extract quotes.


Enter your response:  open in desktop


---------- User ----------
open in desktop
---------- MagenticOneOrchestrator ----------
Please download the video from the provided URL, and then share the local file path here for analysis.


#### Team

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent,critic_agent],termination_condition=text_termination)


In [3]:
result = await team.run(task="Write a short poem about the fall season.")
result

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=137), content="Leaves of amber, gold, and red,  \nWhisper secrets that they spread,  \nCrisp air dances, brisk and clear,  \nHeralding that fall is near.  \n\nPumpkin patches plump and bright,  \nHarvest moon with glowing light,  \nSweaters warm and cider sweet,  \nCrunch of leaves beneath our feet.  \n\nAutumn's palette paints the trees,  \nWith a gentle, cooling breeze,  \nNature's quilt in vibrant hue,  \nBid the summer sun adieu.  \n\nIn this season's fleeting grace,  \nTime slows down its hurried pace,  \nFall’s embrace, a tender call,  \nTo savor beauty in the fall.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=182, completion_tokens=291), content='This poem beautifully captures the essence of the fall s

In [5]:
# when running inside a script, use a async main function and call it from 'asyncio'
await team.reset()
async for message in team.run_stream(task="Write a short poem about the fall season. "):
    if isinstance(message, TaskResult):
        print("Stop Reason: ",message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='Write a short poem about the fall season. ' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=29, completion_tokens=101) content="Leaves cascade in golden showers,  \nWhispering tales through sunlit hours.  \nCrisp air dances with a chill,  \nPainting the landscape with a will.  \n\nPumpkins gleam in amber glow,  \nAs harvest moons begin to show.  \nSweaters wrapped in cozy embrace,  \nWarm hearts gather, sharing grace.  \n\nNature dons her fiery crown,  \nAs autumn's magic settles down.  \nIn every gust and gentle sway,  \nFall weaves wonders day by day.  " type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=147, completion_tokens=162) content='This is a lovely poem that captures the essence of the fall season beautifully. The imagery of "leaves cascade in golden showers" and "pumpkins gleam in amber glow" evokes a warm, inviting picture of autumn. The description of the crisp air and nature\'s t

In [6]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.


---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves cascade in vibrant hues,  
Whispering secrets in the breeze,  
Amber, gold, and crimson fuse,  
A tapestry among the trees.  

Crisp air dances on the skin,  
Pumpkin scents and apple pies,  
Harvest moons begin to spin  
Underneath the starlit skies.  

Sweaters wrap in cozy wool,  
Footsteps crunch on winding trails,  
Nature's brush with colors full,  
Paints a scene where magic dwells.  

Autumn's song resounds anew,  
A fleeting, gentle, sweet adieu.  

---------- critic ----------
Your poem beautifully captures the essence of the fall season with vivid imagery and a rhythmic flow. Here are a few suggestions to enhance it further:

1. **Imagery and Sensory Details**: Consider expanding on the sensory experiences of fall. You've touched on sights and smells; you might want to include sounds or tastes more explicitly to engage the reader more fully.

2. **Consistency in Meter**:

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=119), content="Leaves cascade in vibrant hues,  \nWhispering secrets in the breeze,  \nAmber, gold, and crimson fuse,  \nA tapestry among the trees.  \n\nCrisp air dances on the skin,  \nPumpkin scents and apple pies,  \nHarvest moons begin to spin  \nUnderneath the starlit skies.  \n\nSweaters wrap in cozy wool,  \nFootsteps crunch on winding trails,  \nNature's brush with colors full,  \nPaints a scene where magic dwells.  \n\nAutumn's song resounds anew,  \nA fleeting, gentle, sweet adieu.  \n", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=164, completion_tokens=231), content="Your poem beautifully captures the essence of the fall season with vivid imagery and a rhythmic flow. Here are a few suggestions to enh

In [7]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
In the hush of autumn's call,  
Leaves in vibrant chorus fall,  
Golden light and twilight blend,  
Birch and maple branches bend.  

Chill winds stir the morning mist,  
Nature's palette, artist-kissed,  
Cider warms the waiting hands,  
Harvest time across the lands.  

Sweaters hug against the breeze,  
Whispered tales between the trees,  
Autumn lingers, brief yet bright,  
A fleeting dance of day and night.  


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=562, completion_tokens=101), content="In the hush of autumn's call,  \nLeaves in vibrant chorus fall,  \nGolden light and twilight blend,  \nBirch and maple branches bend.  \n\nChill winds stir the morning mist,  \nNature's palette, artist-kissed,  \nCider warms the waiting hands,  \nHarvest time across the lands.  \n\nSweaters hug against the breeze,  \nWhispered tales between the trees,  \nAutumn lingers, brief yet bright,  \nA fleeting dance of day and night.  ", type='TextMessage')], stop_reason='External termination requested')

In [8]:
await Console(team.run_stream())  # Resume the team to continue the last task.

---------- critic ----------
Your poem beautifully captures the essence and atmosphere of the fall season with rich imagery and a flowing rhythm. Here are a few suggestions to consider for enhancing it further:

1. **Imagery and Sensory Details**: You've beautifully described the visual aspects and atmosphere. Consider incorporating other senses, such as sound or scent, even more vividly to create a fully immersive experience.

2. **Consistency in Meter**: The poem generally flows well, but pay attention to the syllable count for each line to achieve a consistent rhythm. For instance, the lines could be more uniform in length to enhance the rhythmic quality.

3. **Theme Enhancement**: The poem hints at the transient nature of fall, but you could explore this theme slightly more by incorporating metaphors or similes to deepen the reflection on the changing season.

4. **Line Breaks and Structure**: Experimenting with line breaks and stanza structure might add a new layer to the reading 

TaskResult(messages=[TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=893, completion_tokens=263), content="Your poem beautifully captures the essence and atmosphere of the fall season with rich imagery and a flowing rhythm. Here are a few suggestions to consider for enhancing it further:\n\n1. **Imagery and Sensory Details**: You've beautifully described the visual aspects and atmosphere. Consider incorporating other senses, such as sound or scent, even more vividly to create a fully immersive experience.\n\n2. **Consistency in Meter**: The poem generally flows well, but pay attention to the syllable count for each line to achieve a consistent rhythm. For instance, the lines could be more uniform in length to enhance the rhythmic quality.\n\n3. **Theme Enhancement**: The poem hints at the transient nature of fall, but you could explore this theme slightly more by incorporating metaphors or similes to deepen the reflection on the changing season.\n\n4. **Line Breaks